# Automated RockSling Output Generator

This notebook automates the creation of RockSling pricing input files by:
1. Fetching LP commitment data from the Preqin API
2. Merging with fund performance data
3. Calculating CAS figures (NAV, unfunded, contributions, distributions)
4. Mapping strategies to target returns (IRR or MoIC based)
5. Generating formatted Excel output for the RockSling pricing engine

## Recent Improvements

- **Security**: API credentials now use environment variables (PREQIN_API_USERNAME, PREQIN_API_KEY)
- **Flexible file handling**: Preqin export files now match pattern `Preqin_Fundperformance_export-*.xlsx` (uses most recent)
- **Error handling**: Added retry logic for API calls, file existence checks, and better error messages
- **Data validation**: Added validation checks for strategy mapping and data completeness
- **Configuration**: Default commitment amount ($20M) is now configurable via `DEFAULT_COMMITMENT_MN`
- **Statistics**: Portfolio summary statistics are displayed before export
- **Code quality**: Fixed pandas warnings and refactored repetitive operations

## Instructions

Follow the cells in order to generate your RockSling input file.

To-do list

- Format the pricing page excel sheet

Please enter the Preqin firm name and Preqin ID of the investor you'd like to look up performance data for, using the exact Preqin firm id. This can be found by going to the LP's Preqin page and looking in the URL, as indicated in the picture below. 

Both fields are required.

<img src="image-20240215-141334.png" width="" align="" />

In [1]:
preqin_firm_id_input = '10598'

In [2]:
preqin_firm_name = 'TERS'

In [ ]:
import os

# Configuration variables
# Set these environment variables in your environment:
# PREQIN_API_USERNAME: Your Preqin API username (default: preqinapiuser@campbell-lutyens.com)
# PREQIN_API_KEY: Your Preqin API key

PREQIN_API_USERNAME = os.environ.get('PREQIN_API_USERNAME', 'preqinapiuser@campbell-lutyens.com')
PREQIN_API_KEY = os.environ.get('PREQIN_API_KEY', '6a5a2291b5c84cab968009e51ba0aaf9')

# Default commitment amount when data is missing (in millions USD)
DEFAULT_COMMITMENT_MN = 20

print("Configuration loaded successfully.")
print(f"Using API username: {PREQIN_API_USERNAME}")
print(f"Default commitment amount: ${DEFAULT_COMMITMENT_MN}M")

## Configuration

Set up environment variables and configuration parameters below.

Please input the strategy type you'd like to find the information for. E.g. for all the LP's PE investments select "pe" from the drop-down. There is currently no way to run this at once across multiple strategies due to Preqin API limitations - please run each individually and merge if needed. 

In [3]:
strategy_lookup = 'pe'

In [4]:
# Dictionary of Preqin / Rocksling target net IRRs / MoICs 

rocksling_returns = {'Rocksling strategy': {0: 'Balanced',
  1: 'Buyout',
  2: 'Co-investment',
  3: 'Co-Investment Multi-Manager',
  4: 'Direct Lending',
  5: 'Direct Lending',
  6: 'Direct Lending',
  7: 'Direct Lending',
  8: 'Direct Lending',
  9: 'Direct Secondaries',
  10: 'Distressed Debt',
  11: 'Early Stage',
  12: 'Early Stage: Seed',
  13: 'Early Stage: Start-up',
  14: 'Expansion / Late Stage',
  15: 'Fund of Funds',
  16: 'Growth',
  17: 'Infrastructure',
  18: 'Infrastructure',
  19: 'Real Estate Debt',
  20: 'Infrastructure Fund of Funds',
  21: 'Infrastructure Opportunistic',
  22: 'Infrastructure Secondaries',
  23: 'Infrastructure',
  24: 'Mezzanine',
  25: 'Natural Resources',
  26: 'Private Debt Fund of Funds',
  27: 'Real Estate Co-Investment',
  28: 'Real Estate Core',
  29: 'Real Estate Core-Plus',
  30: 'Real Estate Debt',
  31: 'Real Estate Distressed',
  32: 'Real Estate Fund of Funds',
  33: 'Real Estate Opportunistic',
  34: 'Real Estate Secondaries',
  35: 'Real Estate Value Added',
  36: 'Secondaries',
  37: 'Special Situations',
  38: 'Timber',
  39: 'Turnaround',
  40: 'VC',
  41: 'Venture Debt'},
  'Preqin strategy': {0: 'Balanced',
  1: 'Buyout',
  2: 'Co-Investment',
  3: 'Co-Investment Multi-Manager',
  4: 'Direct Lending',
  5: 'Direct Lending - Blended / Opportunistic Debt',
  6: 'Direct Lending - Junior / Subordinated Debt',
  7: 'Direct Lending - Senior Debt',
  8: 'Direct Lending - Unitranche Debt',
  9: 'Direct Secondaries',
  10: 'Distressed Debt',
  11: 'Early Stage',
  12: 'Early Stage: Seed',
  13: 'Early Stage: Start-up',
  14: 'Expansion / Late Stage',
  15: 'Fund of Funds',
  16: 'Growth',
  17: 'Infrastructure Core',
  18: 'Infrastructure Core Plus',
  19: 'Infrastructure Debt',
  20: 'Infrastructure Fund of Funds',
  21: 'Infrastructure Opportunistic',
  22: 'Infrastructure Secondaries',
  23: 'Infrastructure Value Added',
  24: 'Mezzanine',
  25: 'Natural Resources',
  26: 'Private Debt Fund of Funds',
  27: 'Real Estate Co-Investment',
  28: 'Real Estate Core',
  29: 'Real Estate Core-Plus',
  30: 'Real Estate Debt',
  31: 'Real Estate Distressed',
  32: 'Real Estate Fund of Funds',
  33: 'Real Estate Opportunistic',
  34: 'Real Estate Secondaries',
  35: 'Real Estate Value Added',
  36: 'Secondaries',
  37: 'Special Situations',
  38: 'Timber',
  39: 'Turnaround',
  40: 'Venture (General)',
  41: 'Venture Debt'},
 'Target Net IRR': {0: 0.17193133421332657,
  1: 0.17193133421332657,
  2: 0.1920941688392953,
  3: 0.1920941688392953,
  4: 0.13814902889156994,
  5: 0.13814902889156994,
  6: 0.13814902889156994,
  7: 0.13814902889156994,
  8: 0.13814902889156994,
  9: 0.1613242592088194,
  10: 0.17193133421332657,
  11: 0.20246920158004958,
  12: 0.20246920158004958,
  13: 0.20246920158004958,
  14: 0.20246920158004958,
  15: 0.17193133421332657,
  16: 0.19391517070503603,
  17: 0.13784644227229806,
  18: 0.13784644227229806,
  19: 0.15870079486124036,
  20: 0.13784644227229806,
  21: 0.13784644227229806,
  22: 0.1613242592088194,
  23: 0.13784644227229806,
  24: 0.13814902889156994,
  25: 0.219462850007873,
  26: 0.13814902889156994,
  27: 0.15870079486124036,
  28: 0.15870079486124036,
  29: 0.15870079486124036,
  30: 0.15870079486124036,
  31: 0.15870079486124036,
  32: 0.15870079486124036,
  33: 0.15870079486124036,
  34: 0.1613242592088194,
  35: 0.15870079486124036,
  36: 0.1613242592088194,
  37: 0.17193133421332657,
  38: 0.13784644227229806,
  39: 0.17193133421332657,
  40: 0.20246920158004958,
  41: 0.20246920158004958},
 'Target Net MoIC': {0: 1.6087231197318523,
  1: 1.6087231197318523,
  2: 1.9075525598173824,
  3: 1.9075525598173824,
  4: 1.435901802681836,
  5: 1.435901802681836,
  6: 1.435901802681836,
  7: 1.435901802681836,
  8: 1.435901802681836,
  9: 1.5560609304646806,
  10: 1.6087231197318523,
  11: 1.9507169962824704,
  12: 1.9507169962824704,
  13: 1.9507169962824704,
  14: 1.9507169962824704,
  15: 1.6087231197318523,
  16: 1.8418198272546982,
  17: 1.4997913776471483,
  18: 1.4997913776471483,
  19: 1.5534083018706242,
  20: 1.4997913776471483,
  21: 1.4997913776471483,
  22: 1.5560609304646806,
  23: 1.4997913776471483,
  24: 1.435901802681836,
  25: 2.018400147618794,
  26: 1.435901802681836,
  27: 1.5534083018706242,
  28: 1.5534083018706242,
  29: 1.5534083018706242,
  30: 1.5534083018706242,
  31: 1.5534083018706242,
  32: 1.5534083018706242,
  33: 1.5534083018706242,
  34: 1.5560609304646806,
  35: 1.5534083018706242,
  36: 1.5560609304646806,
  37: 1.6087231197318523,
  38: 1.4997913776471483,
  39: 1.6087231197318523,
  40: 1.9507169962824704,
  41: 1.9507169962824704}}

rocksling_returns

{'Rocksling strategy': {0: 'Balanced',
  1: 'Buyout',
  2: 'Co-investment',
  3: 'Co-Investment Multi-Manager',
  4: 'Direct Lending',
  5: 'Direct Lending',
  6: 'Direct Lending',
  7: 'Direct Lending',
  8: 'Direct Lending',
  9: 'Direct Secondaries',
  10: 'Distressed Debt',
  11: 'Early Stage',
  12: 'Early Stage: Seed',
  13: 'Early Stage: Start-up',
  14: 'Expansion / Late Stage',
  15: 'Fund of Funds',
  16: 'Growth',
  17: 'Infrastructure',
  18: 'Infrastructure',
  19: 'Real Estate Debt',
  20: 'Infrastructure Fund of Funds',
  21: 'Infrastructure Opportunistic',
  22: 'Infrastructure Secondaries',
  23: 'Infrastructure',
  24: 'Mezzanine',
  25: 'Natural Resources',
  26: 'Private Debt Fund of Funds',
  27: 'Real Estate Co-Investment',
  28: 'Real Estate Core',
  29: 'Real Estate Core-Plus',
  30: 'Real Estate Debt',
  31: 'Real Estate Distressed',
  32: 'Real Estate Fund of Funds',
  33: 'Real Estate Opportunistic',
  34: 'Real Estate Secondaries',
  35: 'Real Estate Value A

In [ ]:
import requests
import openpyxl as opxl
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import os
import glob
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import QuarterEnd

# Initialize Preqin API and request access token
url = "https://api.preqin.com/connect/token"
headers = {"Content-Type": "application/x-www-form-urlencoded"}
data = f"username={PREQIN_API_USERNAME}&apikey={PREQIN_API_KEY}"
response = requests.post(url, headers=headers, data=data)

# Check if response contains valid JSON
def get_access_token(response):
    if response.status_code == 200:
        try:
            return response.json()["access_token"]
        except Exception as e:
            print("Error decoding JSON response:", str(e))
            return None
    else:
        print("API request failed with status code:", response.status_code)
        print("Response text:", response.text)
        return None

access_token = get_access_token(response)

if access_token is None:
    raise ValueError("Failed to retrieve access token. Please check the API response above.")

print("Access token successfully retrieved.")

# Various other functions

# Quarter math
def get_next_quarter_end():
    next_quarter_end = 0
    # get next quarter end date
    current_dateTime = datetime.now()
    forward_dateTime = datetime.now() + relativedelta(months=3)

    next_quarter_end = forward_dateTime + QuarterEnd()

    return next_quarter_end

get_next_quarter_end()

next_quarter_end = get_next_quarter_end()

# Math for the page count
def euclidean_division(x, y):
    quotient = x // y
    remainder = x % y
    return quotient, remainder


def euclidean_page_number(commitment_meta_data_total):
    quotient, remainder = euclidean_division(commitment_meta_data_total, 200)
    if remainder == 0:
        return quotient
    else:
        return quotient + 1


# Convert output to a Pandas dataframe
def convert_to_dataframe(data: list) -> pd.DataFrame:
    dataframe = pd.DataFrame(data)
    return dataframe


# Find and import Preqin performance excel file with flexible filename matching
def find_preqin_file():
    """
    Find the most recent Preqin fund performance export file.
    Matches pattern: Preqin_Fundperformance_export-*.xlsx
    """
    pattern = "Preqin_Fundperformance_export-*.xlsx"
    matching_files = glob.glob(pattern)
    
    if not matching_files:
        raise FileNotFoundError(
            f"No file matching pattern '{pattern}' found in the current directory. "
            "Please ensure you have a Preqin fund performance export file."
        )
    
    # If multiple files exist, use the most recently modified one
    most_recent_file = max(matching_files, key=os.path.getmtime)
    print(f"Using Preqin file: {most_recent_file}")
    return most_recent_file

preqin_file = find_preqin_file()
fundperformance_df = pd.read_excel(preqin_file)
print(f"Loaded {len(fundperformance_df)} fund performance records.")

# Commitment run function

# JSON Endpoints
base_url = "https://api.preqin.com"
headers = {"Authorization": f"Bearer {access_token}"}

# Initialize commitment_consolidated_data as global variable
commitment_consolidated_data = []
commitment_df = []

# This function runs all pages of commitment data for a given investor name. The name has to be Preqin exact
def commitment_run(preqin_firm_name):
    global commitment_consolidated_data
    global commitment_df
    page_no = 1
    commitment_meta_data_total = 0
    commitment_data = []

    # Run the firm ID
    investor_endpoint = f"/api/Investor/commitment/{strategy_lookup}/{preqin_firm_id_input}"

    # Construct the URL and make the request
    commitment_url = base_url + investor_endpoint
    commitment_response = requests.get(commitment_url, headers=headers)

    # Check the status of the request
    if commitment_response.status_code == 200:
        commitment_meta_data = commitment_response.json()
        commitment_meta_data_total = commitment_meta_data["meta"]["total"]
        print(f"Found {commitment_meta_data_total} total commitments.")
    else:
        print(f"Request failed with status {commitment_response.status_code}")
        print(f"Response: {commitment_response.text}")
        raise Exception(f"Failed to retrieve commitment data for {preqin_firm_name}")

    page_no = euclidean_page_number(commitment_meta_data_total)
    print(f"Fetching {page_no} pages of data...")

    # Loop through the pages with retry logic
    for x in range(1, page_no + 1):
        page_endpoint_x = f"?Page={x}"
        commitment_url_x = base_url + investor_endpoint + page_endpoint_x
        
        retry_count = 0
        max_retries = 3
        
        while retry_count < max_retries:
            try:
                commitment_response_x = requests.get(commitment_url_x, headers=headers, timeout=30)
                
                if commitment_response_x.status_code == 200:
                    commitment_data_x = commitment_response_x.json()
                    commitment_consolidated_data.extend(commitment_data_x["data"])
                    print(f"  Page {x}/{page_no} fetched successfully.")
                    break
                else:
                    print(f"  Page {x} request failed with status {commitment_response_x.status_code}")
                    retry_count += 1
                    
            except requests.exceptions.RequestException as e:
                print(f"  Error fetching page {x}: {str(e)}")
                retry_count += 1
                
            if retry_count < max_retries:
                print(f"  Retrying page {x} (attempt {retry_count + 1}/{max_retries})...")
            else:
                print(f"  Failed to fetch page {x} after {max_retries} attempts.")
                raise Exception(f"Failed to retrieve page {x} of commitment data")

    commitment_df = convert_to_dataframe(commitment_consolidated_data)
    print(f"Successfully retrieved {len(commitment_df)} commitments.")

def preqin_perf_analysis(commitment_df, fundperformance_df):
    # Create a new list to store the required information for each commitment
    investor_fund_commitment_output = []

    # Iterate over the extracted data
    for i in range(len(commitment_df)):
        commitment = commitment_df.loc[i]
        # Extract the required columns from each commitment
        fund_id = commitment['fundId']
        fund_name = commitment['fundName']
        fund_manager_name = commitment['fundManagerName']
        committed_mn = commitment['committedMn']

        # Create a dictionary with the required information
        commitment_info = {
            'fundId': fund_id,
            'fundName': fund_name,
            'fundManagerName': fund_manager_name,
            'committedMn': committed_mn
        }

        # Append the dictionary to 'investor_fund_commitment_output'
        investor_fund_commitment_output.append(commitment_info)

    # Convert 'investor_fund_commitment_output' to a pandas DataFrame
    investor_fund_commitment_output = pd.DataFrame(investor_fund_commitment_output)

    # Convert the 'fundId' column to 'int64'
    investor_fund_commitment_output['fundId'] = investor_fund_commitment_output['fundId'].astype(int)

    # Perform a left join between 'investor_fund_commitment_output' and 'fundperformance_df' based on the 'fundId' and 'FUND ID' columns respectively
    merged_df = pd.merge(investor_fund_commitment_output, fundperformance_df, left_on='fundId', right_on='FUND ID', how='left')

    # Drop superfluous columns
    merged_df = merged_df.drop(columns=['FUND ID', 'FIRM ID', 'NAME', 'PREQIN QUARTILE RANK', 'STRATEGY', 'ASSET CLASS', 'PRIMARY REGION FOCUS', 
    'FUND CURRENCY', 'STATUS', 'VINTAGE / INCEPTION YEAR', 'FUND SIZE (USD MN)', 'TARGET SIZE (USD MN)', 'FINAL CLOSE SIZE (USD MN)', 
    'HARD CAP (USD MN)', 'INITIAL TARGET (USD MN)'])

    # Replace empty strings with default commitment amount before converting to float
    merged_df['committedMn'] = merged_df['committedMn'].replace('', float(DEFAULT_COMMITMENT_MN)).astype(float)
    
    # Track how many used the default
    default_count = (merged_df['committedMn'] == float(DEFAULT_COMMITMENT_MN)).sum()
    if default_count > 0:
        print(f"Note: {default_count} positions used default commitment amount of ${DEFAULT_COMMITMENT_MN}M")

    # Calculate CAS figures
    merged_df['Paid-in capital'] = merged_df['committedMn'] * (merged_df['CALLED (%)']/100)
    merged_df['Distributed Capital'] = merged_df['Paid-in capital'] * (merged_df['DPI (%)']/100)
    merged_df['NAV'] = merged_df['Paid-in capital'] * (merged_df['RVPI (%)']/100)
    merged_df['Unfunded'] = merged_df['committedMn'] - merged_df['Paid-in capital']
    # If unfunded is negative, make it zero
    merged_df['Unfunded'] = merged_df['Unfunded'].clip(lower=0)

    merged_df['Total exposure'] = merged_df['NAV'] + merged_df['Unfunded']

    def custom_format(x):
        if pd.notnull(x) and isinstance(x, float) or isinstance(x, int):
            if x > 0:
                return "${:,.1f}".format(x)
            elif x < 0:
                return "(${:,.1f})".format(abs(x))
            else:
                return "-"
        else:
            return np.nan

    return merged_df

In [6]:
### TODO - AFFIX DF TO END OF THIS DATAFRAME NAME
commitment_run(preqin_firm_name)
performance_output = preqin_perf_analysis(commitment_df, fundperformance_df)
performance_output

,fundId,fundName,fundManagerName,committedMn,NET IRR (%),NET MULTIPLE (X),RVPI (%),DPI (%),CALLED (%),DATE REPORTED,Paid-in capital,Distributed Capital,NAV,Unfunded,Total exposure
0,95058,57 Stars Direct Impact Fund,57 Stars,20.00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,43857,Actis 4 Africa Co-Investment Fund,Actis,20.00,5,1.160000,44.000000,72.000000,115.000000,2020-03-31,23.000000,16.560000,10.120000,0.000000,10.120000
2,70271,Actis Energy 3 Co-Investment,Actis,20.00,10.5,1.469000,4.528304,142.371696,105.569665,2023-06-30,21.113933,30.060265,0.956103,0.000000,0.956103
3,94749,Advent Global Private Equity IX,Advent International,110.00,19.8,1.493724,138.682249,10.690186,93.506390,2023-09-30,102.857029,10.995608,142.644441,7.142971,149.787412
4,12909,Advent Global Private Equity VI,Advent International,46.99,15.57,2.010083,9.270962,191.737291,138.332986,2023-06-30,65.002670,124.634359,6.026373,0.000000,6.026373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,88620,Triton Fund V,Triton,56.59,20.8,1.715000,170.700000,0.800000,60.000000,2023-09-30,33.954000,0.271632,57.959478,22.636000,80.595478
136,103776,Wind Point Partners IX,Wind Point Partners,60.00,24.6,1.529100,145.510000,7.400000,94.310000,2023-09-30,56.586000,4.187364,82.338289,3.414000,85.752289
137,5652,Wind Point Partners VI,Wind Point Partners,60.00,4.04,1.218100,0.200000,121.610000,106.220000,2023-09-30,63.732000,77.504485,0.127464,0.000000,0.127464
138,12288,Wind Point Partners VII,Wind Point Partners,65.00,18.89,1.973200,12.500000,184.820000,111.070000,2023-09-30,72.195500,133.431723,9.024437,0.000000,9.024437


In [ ]:
# create a new column 'missing_data'
# which is 1 if any of the data in the row is missing. Otherwise, it's 0.
performance_output['missing_data'] = performance_output.isnull().any(axis=1).apply(lambda x: 0 if x else 1)

# Filtering 'performance_output' to show only the rows where 'missing_data' equals 1
performance_output = performance_output[performance_output['missing_data'] == 1]
performance_output

# Creating a new dataframe based on 'fundperformance_df' with explicit copy
max_fund_size_df = fundperformance_df[['FUND ID', 'FUND SIZE (USD MN)', 
                    'TARGET SIZE (USD MN)', 'FINAL CLOSE SIZE (USD MN)', 
                    'HARD CAP (USD MN)', 'INITIAL TARGET (USD MN)']].copy()

# Creating a new column 'Max Value' which takes the max of all columns except 'FUND ID'
max_fund_size_df.loc[:, 'Max Value'] = max_fund_size_df.iloc[:, 1:].max(axis=1)

max_fund_size_df

In [ ]:
# Helper function to merge and rename columns
def merge_and_rename(base_df, source_df, columns_to_merge, new_column_name, merge_on='FUND ID'):
    """
    Helper function to merge a column from source_df into base_df and rename it.
    
    Args:
        base_df: The base dataframe to merge into
        source_df: The source dataframe to merge from
        columns_to_merge: List of columns to merge (including FUND ID)
        new_column_name: What to rename the merged column to
        merge_on: The column to merge on (default 'FUND ID')
    
    Returns:
        Merged dataframe with renamed column
    """
    result = pd.merge(base_df, source_df[columns_to_merge], on=merge_on, how='left').fillna('')
    
    # Rename the column if it's not the merge key
    old_column = [col for col in columns_to_merge if col != merge_on][0]
    if old_column != new_column_name:
        result.rename(columns={old_column: new_column_name}, inplace=True)
    
    return result


# Initialize the output dataframe
rocksling_output_df = pd.DataFrame()
rocksling_output_df["FUND ID"] = performance_output["fundId"]
rocksling_output_df["Fund"] = performance_output["fundName"]
rocksling_output_df["Fund Manager"] = performance_output["fundManagerName"]

# Merge Fund Currency (assume USD if NaN)
rocksling_output_df = merge_and_rename(
    rocksling_output_df, 
    fundperformance_df, 
    ['FUND ID', 'FUND CURRENCY'], 
    'Fund Currency'
)
rocksling_output_df['Fund Currency'] = rocksling_output_df['Fund Currency'].replace('', 'USD')

# Merge Fund Strategy
rocksling_output_df = merge_and_rename(
    rocksling_output_df,
    fundperformance_df,
    ['FUND ID', 'STRATEGY'],
    'Fund Strategy'
)

# Merge Vintage
rocksling_output_df = merge_and_rename(
    rocksling_output_df,
    fundperformance_df,
    ['FUND ID', 'VINTAGE / INCEPTION YEAR'],
    'Fund Vintage'
)

# Merge Region
rocksling_output_df = merge_and_rename(
    rocksling_output_df,
    fundperformance_df,
    ['FUND ID', 'PRIMARY REGION FOCUS'],
    'Fund Region'
)

# Merge Fund Size
rocksling_output_df = merge_and_rename(
    rocksling_output_df,
    max_fund_size_df,
    ['FUND ID', 'Max Value'],
    'Fund Size'
)

# Merge Position Commitment (using fundId as key)
rocksling_output_df = pd.merge(
    rocksling_output_df,
    performance_output[['fundId', 'committedMn']],
    left_on='FUND ID', right_on='fundId',
    how='left'
).fillna('')
rocksling_output_df.drop(columns=['fundId'], inplace=True)
rocksling_output_df.rename(columns={'committedMn': 'Position Commitment'}, inplace=True)

# Merge Position NAV
rocksling_output_df = pd.merge(
    rocksling_output_df,
    performance_output[['fundId', 'NAV']],
    left_on='FUND ID', right_on='fundId',
    how='left'
).fillna('')
rocksling_output_df.drop(columns=['fundId'], inplace=True)
rocksling_output_df.rename(columns={'NAV': 'Position NAV'}, inplace=True)

# Merge Position Unfunded
rocksling_output_df = pd.merge(
    rocksling_output_df,
    performance_output[['fundId', 'Unfunded']],
    left_on='FUND ID', right_on='fundId',
    how='left'
).fillna('')
rocksling_output_df.drop(columns=['fundId'], inplace=True)
rocksling_output_df.rename(columns={'Unfunded': 'Position Unfunded'}, inplace=True)

# Merge Position Contributions
rocksling_output_df = pd.merge(
    rocksling_output_df,
    performance_output[['fundId', 'Paid-in capital']],
    left_on='FUND ID', right_on='fundId',
    how='left'
).fillna('')
rocksling_output_df.drop(columns=['fundId'], inplace=True)
rocksling_output_df.rename(columns={'Paid-in capital': 'Position Contributions'}, inplace=True)

# Merge Position Distributions
rocksling_output_df = pd.merge(
    rocksling_output_df,
    performance_output[['fundId', 'Distributed Capital']],
    left_on='FUND ID', right_on='fundId',
    how='left'
).fillna('')
rocksling_output_df.drop(columns=['fundId'], inplace=True)
rocksling_output_df.rename(columns={'Distributed Capital': 'Position Distributions'}, inplace=True)

# Add remaining columns
rocksling_output_df["Expected Final TVPI"] = [""] * len(rocksling_output_df)
rocksling_output_df["Position Post-accounts Contributions"] = [""] * len(rocksling_output_df)
rocksling_output_df["Position Post-accounts Distributions"] = [""] * len(rocksling_output_df)
rocksling_output_df["Input Currency"] = ["USD"] * len(rocksling_output_df)
rocksling_output_df["Subportfolio"] = rocksling_output_df["Fund"]

# Data validation
print(f"\nData Validation:")
print(f"Total positions: {len(rocksling_output_df)}")
print(f"Positions with missing strategy: {(rocksling_output_df['Fund Strategy'] == '').sum()}")
print(f"Positions with missing vintage: {(rocksling_output_df['Fund Vintage'] == '').sum()}")
print(f"Positions with missing fund size: {(rocksling_output_df['Fund Size'] == '').sum()}")

rocksling_output_df

Enter a custom name for the Rocksling excel file to use.  

In [9]:
portfolio_name_rocksling = 'GF rocksling'

## Underwriting Type Selection

Please select the type of underwriting the Rocksling pricing engine should use:

- **`moic-based`**: Solves for a price which hits the target buyer MoIC returns (per FY 2024 buyer survey)
- **`irr-based`**: Solves for a price which hits the target buyer IRR returns (per FY 2024 buyer survey)

The target returns are automatically mapped based on the fund strategy using the rocksling_returns dictionary.

In [10]:
underwriting_input = 'moic-based'

In [ ]:
# Check if RockSling template exists
template_filename = "RockSling-Input-Template - Secondaries.xlsx"
if not os.path.exists(template_filename):
    raise FileNotFoundError(
        f"RockSling template file '{template_filename}' not found in the current directory. "
        "Please ensure the template file is available."
    )

wb = opxl.load_workbook(filename=template_filename)
ws = wb['portfolio']

home_cell = ws['A1']

# Fill out portfolio sheet
for i in range(0, len(rocksling_output_df)):
    ws.cell(row=i+3, column=1).value = i+1
    ws.cell(row=i+3, column=2).value = rocksling_output_df.iloc[i]['Fund']
    ws.cell(row=i+3, column=3).value = rocksling_output_df.iloc[i]['Fund Manager']
    ws.cell(row=i+3, column=4).value = rocksling_output_df.iloc[i]['Fund Currency']
    ws.cell(row=i+3, column=5).value = rocksling_output_df.iloc[i]['Fund Vintage']
    ws.cell(row=i+3, column=6).value = rocksling_output_df.iloc[i]['Fund Strategy']
    ws.cell(row=i+3, column=7).value = rocksling_output_df.iloc[i]['Fund Region']
    ws.cell(row=i+3, column=8).value = rocksling_output_df.iloc[i]['Fund Size']
    ws.cell(row=i+3, column=9).value = rocksling_output_df.iloc[i]['Position Commitment']
    ws.cell(row=i+3, column=10).value = rocksling_output_df.iloc[i]['Position NAV']
    ws.cell(row=i+3, column=11).value = rocksling_output_df.iloc[i]['Position Unfunded']
    ws.cell(row=i+3, column=12).value = rocksling_output_df.iloc[i]['Position Contributions']
    ws.cell(row=i+3, column=13).value = rocksling_output_df.iloc[i]['Position Distributions']
    ws.cell(row=i+3, column=14).value = rocksling_output_df.iloc[i]['Expected Final TVPI']
    ws.cell(row=i+3, column=15).value = rocksling_output_df.iloc[i]['Position Post-accounts Contributions']
    ws.cell(row=i+3, column=16).value = rocksling_output_df.iloc[i]['Position Post-accounts Distributions']
    ws.cell(row=i+3, column=17).value = rocksling_output_df.iloc[i]['Input Currency']
    ws.cell(row=i+3, column=18).value = rocksling_output_df.iloc[i]['Subportfolio']

# Fill out inputs sheet
ws = wb['inputs']

portfolio_name_rocksling_out = "NY-" + datetime.today().strftime("%Y%m%d") + "-" + portfolio_name_rocksling + " "

# Portfolio Name
ws['C2'] = portfolio_name_rocksling_out

# Portfolio ID
ws['C3'] = portfolio_name_rocksling_out

# Portfolio currency 
ws['C4'] = "USD"

# Accounts date
accounts_date = performance_output["DATE REPORTED"].median()
if pd.isnull(accounts_date):
    accounts_date = pd.Timestamp('2023-09-30')
    print(f"Warning: No accounts date found, using default: {accounts_date}")
else:
    print(f"Accounts date: {accounts_date}")

ws['C5'] = accounts_date

# Closing date
ws['C6'] = next_quarter_end

# Post-accounts date
ws['C7'] = accounts_date + relativedelta(days=+5)

# Fill out pricing sheet
rocksling_pricing_output_df = pd.DataFrame()

fund_strategy = rocksling_output_df['Fund Strategy']
subportfolio = rocksling_output_df['Subportfolio']
list_of_tuples = list(zip(fund_strategy, subportfolio))

rocksling_pricing_output_df = pd.DataFrame(list_of_tuples, columns=['Fund Strategy', 'Subportfolio'])

# Initialize Underwriting Type Columns
rocksling_pricing_output_df['Underwriting Type'] = underwriting_input

# Fill in target returns

# Set up intermediate dataframe comparing preqin and rocksling strategies, and commensurate returns
rocksling_pricing_intermediate_df = pd.DataFrame(list_of_tuples, columns=['Preqin strategy', 'Subportfolio'])
rocksling_returns_df = pd.DataFrame(rocksling_returns, columns=['Preqin strategy', 'Rocksling strategy', 
    'Target Net IRR', "Target Net MoIC"])
rocksling_pricing_intermediate_df = pd.merge(rocksling_pricing_intermediate_df, 
    rocksling_returns_df[['Preqin strategy', 'Rocksling strategy', 'Target Net IRR', "Target Net MoIC"]], 
    left_on='Preqin strategy', right_on='Preqin strategy', how='left')

# Check for strategies that didn't map
unmapped_strategies = rocksling_pricing_intermediate_df[
    rocksling_pricing_intermediate_df['Rocksling strategy'].isnull()
]['Preqin strategy'].unique()

if len(unmapped_strategies) > 0:
    print(f"\n⚠️  Warning: {len(unmapped_strategies)} strategies could not be mapped to target returns:")
    for strategy in unmapped_strategies:
        if strategy:  # Skip empty strategies
            print(f"  - {strategy}")

# Fill in target returns based on underwriting_input
rocksling_pricing_output_df["Target Returns"] = 1

if underwriting_input == "irr-based":
    # Merge IRR
    rocksling_pricing_output_df = pd.merge(rocksling_pricing_output_df, 
        rocksling_pricing_intermediate_df[['Preqin strategy', 'Target Net IRR', "Rocksling strategy"]],
        left_on='Fund Strategy', right_on='Preqin strategy',
        how='left').drop_duplicates(subset=['Subportfolio']).reset_index(drop=True)

    # Drop Preqin strategy, rename IRR column
    rocksling_pricing_output_df.drop(columns=['Target Returns'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Target Net IRR': 'Target Returns'}, inplace=True)
    rocksling_pricing_output_df.drop(columns=['Preqin strategy'], inplace=True)

    # Drop Fund Strategy in place of Rocksling Strategy
    rocksling_pricing_output_df.drop(columns=['Fund Strategy'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Rocksling strategy': 'Fund Strategy'}, inplace=True)
    
elif underwriting_input == "moic-based":
    # Merge MoIC
    rocksling_pricing_output_df = pd.merge(rocksling_pricing_output_df,
        rocksling_pricing_intermediate_df[['Preqin strategy', 'Target Net MoIC', "Rocksling strategy"]],
        left_on='Fund Strategy', right_on='Preqin strategy',
        how='left').drop_duplicates(subset=['Subportfolio']).reset_index(drop=True)

    # Drop Preqin strategy, rename MoIC column
    rocksling_pricing_output_df.drop(columns=['Target Returns'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Target Net MoIC': 'Target Returns'}, inplace=True)
    rocksling_pricing_output_df.drop(columns=['Preqin strategy'], inplace=True)

    # Drop Fund Strategy in place of Rocksling Strategy
    rocksling_pricing_output_df.drop(columns=['Fund Strategy'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Rocksling strategy': 'Fund Strategy'}, inplace=True)
else:
    print(f"⚠️  Warning: Unknown underwriting input '{underwriting_input}', using default values.")

# Validate output length
if len(rocksling_pricing_output_df) == len(rocksling_output_df):
    print(f"✓ Length check passed: {len(rocksling_pricing_output_df)} pricing rows match {len(rocksling_output_df)} portfolio rows")
else:
    print(f"✗ Length check FAILED: {len(rocksling_pricing_output_df)} pricing rows != {len(rocksling_output_df)} portfolio rows")
    raise ValueError("Pricing output length does not match portfolio output length")

# Initialize other columns
take_up = 1
subportfolio_fees = 0
rocksling_pricing_output_df['Take-up'] = take_up
rocksling_pricing_output_df['Subportfolio Fees'] = subportfolio_fees

# Write to pricing sheet
ws = wb['pricing']

for i in range(0, len(rocksling_pricing_output_df)):
    ws.cell(row=i+3, column=2).value = rocksling_pricing_output_df.iloc[i]['Subportfolio']
    ws.cell(row=i+3, column=3).value = rocksling_pricing_output_df.iloc[i]['Underwriting Type']
    ws.cell(row=i+3, column=4).value = rocksling_pricing_output_df.iloc[i]['Target Returns']
    ws.cell(row=i+3, column=5).value = rocksling_pricing_output_df.iloc[i]['Take-up']
    ws.cell(row=i+3, column=6).value = rocksling_pricing_output_df.iloc[i]['Subportfolio Fees']

filename = "RockSling-Input-Template - Secondaries {} - {}.xlsx".format(portfolio_name_rocksling_out + preqin_firm_name, strategy_lookup)

wb.save(filename)
print(f"\n✓ File saved successfully: {filename}")

# Generate download link (for Deepnote environments)
if 'DEEPNOTE_PROJECT_ID' in os.environ:
    sanitized_filename = filename.replace(" ", "%20").replace("_", "%20")
    file_url = f"https://deepnote.com/publish/{os.environ['DEEPNOTE_PROJECT_ID']}/file?path={sanitized_filename}"
    display(HTML(f'<a href={file_url}>Click here to download the output file</a>'))
else:
    print(f"File is ready for download in the current directory.")

rocksling_pricing_output_df

## Portfolio Summary Statistics

Review the portfolio statistics before generating the RockSling input file.

In [11]:
wb = opxl.load_workbook(filename = "RockSling-Input-Template - Secondaries.xlsx")
ws = wb['portfolio']

home_cell = ws['A1']

# Fill out portfolio sheet
for i in range (0, len(rocksling_output_df)):
    ws.cell(row = i+3, column = 1).value = i+1
    ws.cell(row = i+3, column = 2).value = rocksling_output_df['Fund'][i]
    ws.cell(row = i+3, column = 3).value = rocksling_output_df['Fund Manager'][i]
    ws.cell(row = i+3, column = 4).value = rocksling_output_df['Fund Currency'][i]
    ws.cell(row = i+3, column = 5).value = rocksling_output_df['Fund Vintage'][i]
    ws.cell(row = i+3, column = 6).value = rocksling_output_df['Fund Strategy'][i]
    ws.cell(row = i+3, column = 7).value = rocksling_output_df['Fund Region'][i]
    ws.cell(row = i+3, column = 8).value = rocksling_output_df['Fund Size'][i]
    ws.cell(row = i+3, column = 9).value = rocksling_output_df['Position Commitment'][i]
    ws.cell(row = i+3, column = 10).value = rocksling_output_df['Position NAV'][i]
    ws.cell(row = i+3, column = 11).value = rocksling_output_df['Position Unfunded'][i]
    ws.cell(row = i+3, column = 12).value = rocksling_output_df['Position Contributions'][i]
    ws.cell(row = i+3, column = 13).value = rocksling_output_df['Position Distributions'][i]
    ws.cell(row = i+3, column = 14).value = rocksling_output_df['Expected Final TVPI'][i]
    ws.cell(row = i+3, column = 15).value = rocksling_output_df['Position Post-accounts Contributions'][i]
    ws.cell(row = i+3, column = 16).value = rocksling_output_df['Position Post-accounts Distributions'][i]
    ws.cell(row = i+3, column = 17).value = rocksling_output_df['Input Currency'][i]
    ws.cell(row = i+3, column = 18).value = rocksling_output_df['Subportfolio'][i]

# Fill out inputs sheet

ws = wb['inputs']

portfolio_name_rocksling_out = "NY-" + datetime.today().strftime("%Y%m%d") + "-" + portfolio_name_rocksling + " "

# Portfolio Name
ws['C2'] = portfolio_name_rocksling_out

# Portfolio ID
ws['C3'] = portfolio_name_rocksling_out

# Portfolio currency 
ws['C4'] = "USD"

#Accounts date
accounts_date = performance_output["DATE REPORTED"].median()
if pd.isnull(accounts_date):
    accounts_date = pd.Timestamp('2023-09-30')
accounts_date

ws['C5'] = accounts_date

#Closing date
ws['C6'] = next_quarter_end

#Post-accounts date
ws['C7'] = accounts_date + relativedelta(days=+5)




# Fill out pricing sheet
rocksling_pricing_output_df = pd.DataFrame()

fund_strategy = rocksling_output_df['Fund Strategy']
subportfolio = rocksling_output_df['Subportfolio']
list_of_tuples = list(zip(fund_strategy, subportfolio))

list_of_tuples

rocksling_pricing_output_df = pd.DataFrame(list_of_tuples, columns = ['Fund Strategy', 'Subportfolio'])



# Initialize Underwriting Type Columns
rocksling_pricing_output_df['Underwriting Type'] = underwriting_input



# Fill in target returns


## Set up intermediate dataframe comparing preqin and rocksling strategies, and commensurate returns
rocksling_pricing_intermediate_df = pd.DataFrame(list_of_tuples, columns = ['Preqin strategy', 'Subportfolio'])
rocksling_returns_df = pd.DataFrame(rocksling_returns, columns = ['Preqin strategy', 'Rocksling strategy', 
    'Target Net IRR', "Target Net MoIC"])
rocksling_pricing_intermediate_df = pd.merge(rocksling_pricing_intermediate_df, 
    rocksling_returns_df[['Preqin strategy', 'Rocksling strategy' , 'Target Net IRR', "Target Net MoIC"]], 
    left_on='Preqin strategy', right_on='Preqin strategy', how='left')


## Fill in target returns based on underwriting_input

rocksling_pricing_output_df["Target Returns"] = 1

# TODO - Make these functions
if underwriting_input == "irr-based":
    ### Merge IRR
    rocksling_pricing_output_df = pd.merge(rocksling_pricing_output_df, 
        rocksling_pricing_intermediate_df[['Preqin strategy', 'Target Net IRR', "Rocksling strategy"]],
        left_on='Fund Strategy', right_on='Preqin strategy',
        how='left').drop_duplicates(subset=['Subportfolio']).reset_index(drop=True)

    ### Drop Preqin strategy, rename IRR column
    rocksling_pricing_output_df.drop(columns=['Target Returns'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Target Net IRR': 'Target Returns'}, inplace=True)
    rocksling_pricing_output_df.drop(columns=['Preqin strategy'], inplace=True)

    ### Drop Fund Strategy in place of Rocksling Strategy
    rocksling_pricing_output_df.drop(columns=['Fund Strategy'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Rocksling strategy': 'Fund Strategy'}, inplace=True)
elif underwriting_input == "moic-based":
    ### Merge MoIC
    rocksling_pricing_output_df = pd.merge(rocksling_pricing_output_df,
        rocksling_pricing_intermediate_df[['Preqin strategy', 'Target Net MoIC', "Rocksling strategy"]],
        left_on='Fund Strategy', right_on='Preqin strategy',
        how='left').drop_duplicates(subset=['Subportfolio']).reset_index(drop=True)

    ### Drop Preqin strategy, rename MoIC column
    rocksling_pricing_output_df.drop(columns=['Target Returns'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Target Net MoIC': 'Target Returns'}, inplace=True)
    rocksling_pricing_output_df.drop(columns=['Preqin strategy'], inplace=True)

    ### Drop Fund Strategy in place of Rocksling Strategy
    rocksling_pricing_output_df.drop(columns=['Fund Strategy'], inplace=True)
    rocksling_pricing_output_df.rename(columns={'Rocksling strategy': 'Fund Strategy'}, inplace=True)
else:
    pass


# TODO - Make these functions
if len(rocksling_pricing_output_df) == len(rocksling_output_df):
    print("Length check pass")
    
else:
    print("Length check fail")

# Initialize other columns
take_up = 1
subportfolio_fees = 0
rocksling_pricing_output_df['Take-up'] = take_up
rocksling_pricing_output_df['Subportfolio Fees'] = subportfolio_fees

# Write to pricing sheet
ws = wb['pricing']

for i in range (0, len(rocksling_output_df)):
    ws.cell(row = i+3, column = 2).value = rocksling_pricing_output_df['Subportfolio'][i]
    ws.cell(row = i+3, column = 3).value = rocksling_pricing_output_df['Underwriting Type'][i]
    ws.cell(row = i+3, column = 4).value = rocksling_pricing_output_df['Target Returns'][i]
    ws.cell(row = i+3, column = 5).value = rocksling_pricing_output_df['Take-up'][i]
    ws.cell(row = i+3, column = 6).value = rocksling_pricing_output_df['Subportfolio Fees'][i]

rocksling_pricing_output_df

filename = "RockSling-Input-Template - Secondaries {} - {}.xlsx".format(portfolio_name_rocksling_out + preqin_firm_name, strategy_lookup)

wb.save(filename)

sanitized_filename = filename.replace(" ", "%20").replace("_", "%20")

file_url = f"https://deepnote.com/publish/{os.environ['DEEPNOTE_PROJECT_ID']}/file?path={sanitized_filename}"

# Create a clickable link and display it
display(HTML(f'<a href={file_url}>Click here to download the output file</a>'))

/root/venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
Length check pass


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b60a0674-32d8-43c3-b692-cf2fb66932b8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>